In [1]:
import argparse
import logging
import os
import random
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torchmetrics
from pathlib import Path
from torch import optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import numpy as np
import time
import math
import lightning.pytorch as pl

import wandb
from evaluate import evaluate
from unet import UNet
from unet.unet_model_lightning import UNetLightning
from utils.data_loading import BasicDataset, CarvanaDataset
from utils.dice_score import dice_loss
from eevaluate import etest
from etrainer import train_model, config_data
from etrainerfunctions import rotate
import etrainer

C:\Users\jjkjj\anaconda3\envs\equivariant\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [2]:
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=torch.device('cpu')
epsilon=0

sizeup = torchvision.transforms.Resize((550,550), interpolation=TF.InterpolationMode.NEAREST_EXACT)
sizedown=torchvision.transforms.Resize((450,450), interpolation=TF.InterpolationMode.NEAREST_EXACT)
resize=torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.NEAREST_EXACT)
deformation = torch.tensor([[0,0,0],[0,.5,0],[0,0,0]]).to(device=device, dtype=torch.float32)
deformation=torch.unsqueeze(deformation, dim=0)
deformation= torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.BICUBIC)(deformation)
ydeformation =torch.full([1,512,512], 0, device=device, dtype=torch.float32)
deformation = torch.stack([deformation, ydeformation], dim=3)
deform = lambda tensor : TF.elastic_transform(tensor, deformation, TF.InterpolationMode.NEAREST, 0.0)


upscale=torchvision.transforms.Compose([sizeup,resize])
etransform=torchvision.transforms.Compose([sizedown,resize])
shrinkcrop=etrainer.compose(etrainer.pad24,sizedown)

def shift(x, shiftnum=1, axis=-1):
    x=torch.transpose(x, axis, -1)
    if shiftnum == 0:
        padded = x
    elif shiftnum > 0:
        #paddings = (0, shift, 0, 0, 0, 0)
        paddings = [0 for j in range(2*len(tuple(x.shape)))]
        paddings[1]=shiftnum
        paddings=tuple(paddings)
        padded = nn.functional.pad(x[..., shiftnum:], paddings)
    elif shiftnum < 0:
        #paddings = (-shift, 0, 0, 0, 0, 0)
        paddings = [0 for j in range(2*len(tuple(x.shape)))]
        paddings[0]=-shiftnum
        paddings=tuple(paddings)
        padded = nn.functional.pad(x[..., :shiftnum], paddings)
    else:
        raise ValueError
    return torch.transpose(padded, axis,-1)
def randshift(x):
    shiftnum = random.randint(-6,6)
    axis = random.randint(-2,-1)
    return shift(x, shiftnum, axis)
#This is for scaling
#efunctions=[[etransform, etransform, epsilon], [upscale,upscale,epsilon]] 
efunctions=[[torchvision.transforms.RandomRotation(10), torchvision.transforms.RandomRotation(10),epsilon]]
#efunctions=efunctions+[[lambda x : shift(x, shiftnum, axis), lambda x : shift(x, shiftnum, axis), epsilon] for shiftnum in range(-1,1,2) for axis in range(-1,1,2)]+[[etransform, etransform, epsilon], [upscale,upscale,epsilon]]
#efunctions += [[randshift, randshift, epsilon]]
#efunctions = efunctions+[[deform,deform,epsilon]]
#efunctions = efunctions + [[rotate(90), rotate(90),0]]
#efunctions = efunctions + [[torchvision.transforms.ElasticTransform(interpolation=TF.InterpolationMode.NEAREST), torchvision.transforms.ElasticTransform(interpolation=TF.InterpolationMode.NEAREST), epsilon]]
"""
for x in range(-1,1):
    for y in range(-1,1):
        for i in range(3):
            for j in range(3):
                for a in range(3):
                    for b in range(3):
                        deformationx = torch.tensor([[x if i==k else 0 for k in range(3)] if j==l else [0,0,0] for l in range(3)]).to(device=device, dtype=torch.float32)
                        deformationy = torch.tensor([[y if a==k else 0 for k in range(3)] if b==l else [0,0,0] for l in range(3)]).to(device=device, dtype=torch.float32)
                        deformationx=torch.unsqueeze(deformationx, dim=0)
                        deformationy=torch.unsqueeze(deformationy, dim=0)
                        deformationx= torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.BICUBIC)(deformationx)
                        deformationy= torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.BICUBIC)(deformationy)
                        deformation = torch.stack([deformationx, deformationy], dim=3)
                        deform = lambda tensor : TF.elastic_transform(tensor, deformation, TF.InterpolationMode.NEAREST, 0.0)
                        efunctions.append([deform,deform,0])
"""
args = {'epochs' : 50,
        'batch_size' : 8,
        'amp' : True,
        'bilinear' : False,
        'classes' : 3,
        'learning_rate' : 1e-6,
        'load': False,
        #'load' : "C:\\Users\\jjkjj\\Equivariant\\EquivariantUNet\\bumbling-sponge-27_checkpoints\\checkpoint_epoch121.pth",
        'class weights' : [1,1,3],
        'epochbreaks' : False,
        'break_length' : 5,
        'etransforms' : efunctions,
        'equivariance_measure' : 'l1',
        'equivariant' : False,
        'eqerror' : False,
        'augmented' : 'rangle',
        'Linf' : False,
        'eqweight' : .1,
        'n' : 1,
        'debugging' : False,
        'in_channels' : 3,
        'wandb_project' : 'Equivariant UNet',
        'test_on_epoch_end' : True,
        'test augmented' : False,
        'test augment' : 'model transforms',
        'save_checkpoint' : False,
        'eqweight_scheduler' : False,
        'eqweight_decay' : 1.1,
        'lr_scheduler' : 'cyclic',
        'min_lr' : 1e-9,
        'max_lr' : 1.5e-4,
        'product_loss' : False,
        'C1norm' : False,
        'C1weight' : 1,
        'Oxford' : True,
        'HeLa' : False,
       }
if args['equivariant'] != args['eqerror']:
    print('Equivariant and eqerror are different are you sure?')

model = UNet(args['in_channels'], args['classes'], **args).to(device=device, dtype = torch.float32)
model = model.to(memory_format=torch.channels_last)
angle = 5

#model = UNetLightning(args['in_channels'], args['classes'], **args).to(device=device)


if args['load']:
    state_dict = torch.load(args['load'], map_location=device)
    #del state_dict['mask_values']
    model.load_state_dict(state_dict)
    logging.info(f'Model loaded from {args["load"]}')
print(model.n)
print(model.Linf)
#print([param for param in list(model.parameters())])


1
False


In [7]:
etest(model, config_data(HeLa=args['HeLa'], Oxford=args['Oxford'], split='test',
                         augmented=args['test augmented'], aug_transforms=[[rotate(angle, fill=1),rotate(angle, fill=1),0,1]]), device, True,
       epoch=1, experiment_started=False,
                         etransforms=[[rotate(angle, fill = 1),rotate(angle, fill=1),0,1]],
                         test_augment = args['test augment'], angle = angle, class_weights = None, wandb_project = 'Equivariant UNet')

KeyboardInterrupt: 

In [ ]:
train_loader = config_data(**args)
train_model(model, device, train_loader, **args)

wandb: Currently logged in as: jjkjjk23. Use `wandb login --relogin` to force relogin


Epoch 1/50: 100%|█| 3480/3480 [03:21<00:00, 17.31img/s, Cumulative Dice=[0.7646768675453361, 0.8417441009104937, 0.3478
Epoch 2/50: 100%|█| 3480/3480 [03:42<00:00, 15.67img/s, Cumulative Dice=[0.8419741481200032, 0.8944961483451142, 0.5281
Epoch 3/50: 100%|█| 3480/3480 [03:54<00:00, 14.85img/s, Cumulative Dice=[0.8309374424232834, 0.8887414850037674, 0.5198
Epoch 4/50: 100%|█| 3480/3480 [03:50<00:00, 15.09img/s, Cumulative Dice=[0.8445855516126786, 0.9045857513087919, 0.5459
Epoch 5/50: 100%|█| 3480/3480 [03:49<00:00, 15.15img/s, Cumulative Dice=[0.8826471347918455, 0.92305296179892, 0.608765
Epoch 6/50: 100%|█| 3480/3480 [03:49<00:00, 15.14img/s, Cumulative Dice=[0.8461500255540869, 0.9001461151002468, 0.5523
Epoch 7/50: 100%|█| 3480/3480 [03:50<00:00, 15.07img/s, Cumulative Dice=[0.8965620878099025, 0.9318929629764338, 0.6326
Epoch 8/50: 100%|█| 3480/3480 [03:50<00:00, 15.09img/s, Cumulative Dice=[0.8651940781494667, 0.9108806690950503, 0.5751
Epoch 9/50: 100%|█| 3480/3480 [03:51<00:

In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "caching_allocator"
train_loader = config_data(HeLa=True, Oxford=False, **args)
trainer = pl.Trainer(max_epochs=1, accelerator='gpu')
trainer.fit(model, train_dataloaders=train_loader)

In [3]:
import PIL
import torch
import torchvision

class HeLaDataset(torch.utils.data.Dataset):
    def __init__(self, train_file,label_file, n_images, transform=None, target_transform=None):
        super().__init__()
        self.images=PIL.Image.open(train_file)
        self.images.load()
        self.labels=PIL.Image.open(label_file)
        self.labels.load()
        #self.tensor=torchvision.transforms.PILToTensor(PIL.Image.open(file))
        self.n_images=n_images
        self.transform=transform
        self.target_transform=target_transform
        
    def __getitem__(self, idx):
        print(idx)
        self.images.seek(idx)
        if self.transform:
            return self.transform(torch.from_numpy(np.array(self.images))), self.target_transform(torch.from_numpy(np.array(self.labels)))
        else:
            return torch.from_numpy(np.array(self.images)), torch.from_numpy(np.array(self.labels))

        #return self
    def __len__(self):
        return self.n_images


In [2]:
dataset= HeLaDataset(f"C:\\Users\\jjkjj\\Equivariant\\ISBI-2012-challenge\\train-volume.tif","C:\\Users\\jjkjj\\Equivariant\\ISBI-2012-challenge\\train-labels.tif", 30)
for j, image in enumerate(dataset):
    print(image)

    

NameError: name 'HeLaDataset' is not defined

In [3]:
#
time0=time.time()
#@torch.inference_mode()
def timemodel():
    #model.eval()
    for batch in config_data(HeLa=True, Oxford=False, augmented='randcombo'):
        image, label = batch
        imager= torchvision.transforms.ToPILImage()
        image = imager(torch.squeeze(image, dim=0))
        label = imager(torch.squeeze(label, dim=0))
        image.show()
        label.show()
    return None
"""
        x, y = batch
        x=x.to(device=self.device)
        y=y.to(device=self.device)
        y_hat = self(x).to(device=self.device)
        #randomval = torch.from_numpy(np.random.default_rng().random(size=(1,1,512,512))).to(torch.float32)
        randomval=etrainer.sampler((1,512,512),n=1, cuda=False)
        loss = F.binary_cross_entropy_with_logits(y_hat, y.to(torch.float32))
        if self.equivariant:
            for f in self.etransforms:
                loss+= torch.mean(torch.abs(self(f[1](randomval))-f[0](self(randomval))))
        return loss
"""
timemodel()


KeyboardInterrupt: 

In [6]:
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=torch.device('cpu')
epsilon=0

sizeup = torchvision.transforms.Resize((550,550), interpolation=TF.InterpolationMode.NEAREST_EXACT)
sizedown=torchvision.transforms.Resize((450,450), interpolation=TF.InterpolationMode.NEAREST_EXACT)
resize=torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.NEAREST_EXACT)
deformation = torch.tensor([[0,0,0],[0,.5,0],[0,0,0]]).to(device=device, dtype=torch.float32)
deformation=torch.unsqueeze(deformation, dim=0)
deformation= torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.BICUBIC)(deformation)
ydeformation =torch.full([1,512,512], 0, device=device, dtype=torch.float32)
deformation = torch.stack([deformation, ydeformation], dim=3)
deform = lambda tensor : TF.elastic_transform(tensor, deformation, TF.InterpolationMode.NEAREST, 0.0)

def rotate(angle):
    return lambda inputs : torchvision.transforms.functional.rotate(inputs, angle)


upscale=torchvision.transforms.Compose([sizeup,resize])
etransform=torchvision.transforms.Compose([sizedown,resize])
shrinkcrop=etrainer.compose(etrainer.pad24,sizedown)

def shift(x, shiftnum=1, axis=-1):
    x=torch.transpose(x, axis, -1)
    if shiftnum == 0:
        padded = x
    elif shiftnum > 0:
        #paddings = (0, shift, 0, 0, 0, 0)
        paddings = [0 for j in range(2*len(tuple(x.shape)))]
        paddings[1]=shiftnum
        paddings=tuple(paddings)
        padded = nn.functional.pad(x[..., shiftnum:], paddings)
    elif shiftnum < 0:
        #paddings = (-shift, 0, 0, 0, 0, 0)
        paddings = [0 for j in range(2*len(tuple(x.shape)))]
        paddings[0]=-shiftnum
        paddings=tuple(paddings)
        padded = nn.functional.pad(x[..., :shiftnum], paddings)
    else:
        raise ValueError
    return torch.transpose(padded, axis,-1)
def randshift(x):
    shiftnum = random.randint(-6,6)
    axis = random.randint(-2,-1)
    return shift(x, shiftnum, axis)
#This is for scaling
#efunctions=[[etransform, etransform, epsilon], [upscale,upscale,epsilon]] 
efunctions=[[torchvision.transforms.RandomRotation(10), torchvision.transforms.RandomRotation(10),epsilon]]
#efunctions=efunctions+[[lambda x : shift(x, shiftnum, axis), lambda x : shift(x, shiftnum, axis), epsilon] for shiftnum in range(-1,1,2) for axis in range(-1,1,2)]+[[etransform, etransform, epsilon], [upscale,upscale,epsilon]]
#efunctions += [[randshift, randshift, epsilon]]
#efunctions = efunctions+[[deform,deform,epsilon]]
#efunctions = efunctions + [[rotate(90), rotate(90),0]]
#efunctions = efunctions + [[torchvision.transforms.ElasticTransform(interpolation=TF.InterpolationMode.NEAREST), torchvision.transforms.ElasticTransform(interpolation=TF.InterpolationMode.NEAREST), epsilon]]
"""
for x in range(-1,1):
    for y in range(-1,1):
        for i in range(3):
            for j in range(3):
                for a in range(3):
                    for b in range(3):
                        deformationx = torch.tensor([[x if i==k else 0 for k in range(3)] if j==l else [0,0,0] for l in range(3)]).to(device=device, dtype=torch.float32)
                        deformationy = torch.tensor([[y if a==k else 0 for k in range(3)] if b==l else [0,0,0] for l in range(3)]).to(device=device, dtype=torch.float32)
                        deformationx=torch.unsqueeze(deformationx, dim=0)
                        deformationy=torch.unsqueeze(deformationy, dim=0)
                        deformationx= torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.BICUBIC)(deformationx)
                        deformationy= torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.BICUBIC)(deformationy)
                        deformation = torch.stack([deformationx, deformationy], dim=3)
                        deform = lambda tensor : TF.elastic_transform(tensor, deformation, TF.InterpolationMode.NEAREST, 0.0)
                        efunctions.append([deform,deform,0])
"""
args = {'epochs' : 100,
        'batch_size' : 1,
        'amp' : True,
        'bilinear' : False,
        'classes' : 1,
        'learning_rate' : 1e-6,
        'load': False,
        #'load' : "C:\\Users\\jjkjj\\Equivariant\\EquivariantUNet\\bumbling-sponge-27_checkpoints\\checkpoint_epoch121.pth",
        'class weights' : [1,1,3],
        'epochbreaks' : False,
        'break_length' : 5,
        'etransforms' : efunctions,
        'equivariance_measure' : 'l1',
        'equivariant' : True,
        'eqerror' : True,
        'augmented' : 'rangle',
        'Linf' : False,
        'eqweight' : 100,
        'n' : 1,
        'debugging' : False,
        'in_channels' : 1,
        'wandb_project' : 'HeLa EUNet',
        'test_on_epoch_end' : True,
        'test augmented' : 'True no identity',
        'test augment' : 'fixed rotations',
        'save_checkpoint' : True,
        'eqweight_scheduler' : False,
        'eqweight_decay' : 1.1,
        'lr_scheduler' : 'cyclic',
        'min_lr' : 1e-9,
        'max_lr' : 1.5e-4,
        'product_loss' : False
        'C1norm' : True
        'C1weight' : 1
       }
if args['equivariant'] != args['eqerror']:
    print('Equivariant and eqerror are different are you sure?')

model = UNet(args['in_channels'], args['classes'], **args).to(device=device)
model = model.to(memory_format=torch.channels_last)
etest(model, config_data(HeLa=args['HeLa'], Oxford=args['Oxford'], split='test', augmented=args['test augmented'], 
    aug_transforms=[[rotate(angle),rotate(angle),0,1]], **kwargs), device, amp, run_id=experiment.id, epoch=epoch,
    experiment_started=False, 
    etransforms=[[rotate(angle),rotate(angle),0,1]], test_augment = kwargs['test augment'], angle = angle)

#model = UNetLightning(args['in_channels'], args['classes'], **args).to(device=device)


if args['load']:
    state_dict = torch.load(args['load'], map_location=device)
    #del state_dict['mask_values']
    model.load_state_dict(state_dict)
    logging.info(f'Model loaded from {args["load"]}')
print(model.n)
print(model.Linf)
print(model.state_dict()['inc.double_conv.0.weight'])


1
False
torch.Size([64, 1, 3, 3])


In [ ]:
for j in range(10):
    model = UNet(args['in_channels'], args['classes'], **args2).to(device=device)
    model = model.to(memory_format=torch.channels_last)
    train_loader = config_data(HeLa=True, Oxford=False, **args2)
    train_model(model, device, train_loader, **args2)

Angle -10 Test Loss,██▇▄▂▃▃▂▂▂▁▂▁▁▁▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▂▁
Angle -20 Test Loss,██▇▄▃▃▃▂▂▃▂▂▂▁▁▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▂▂▁▁▁▂▃▁▂▂
Angle -5 Test Loss,███▄▂▃▃▂▂▂▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▂▁
Angle 0 Test Loss,███▄▂▃▃▂▂▂▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▁▁▁
Angle 10 Test Loss,██▇▄▂▃▃▂▂▂▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁
Angle 20 Test Loss,██▇▄▃▃▃▂▂▃▂▃▂▁▁▂▂▂▁▂▂▂▂▂▂▁▁▁▁▂▂▂▁▂▁▂▂▁▂▁
Angle 5 Test Loss,███▄▂▃▃▂▁▂▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▂▁
Equivariance Weight,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Random Equivariance Error,▁▁▁▁▂▁▁▁▂▂▁▂▃▁▂▃▁▄▃▇▅▆▆▂▅▂▂█▆▂▄▄▁▂▄▁█▇▃▄
Test Equivariance Error,▂▁▁▃▄▄▅▅▅▅▅▆▆▅▆▆▄▅▆▆▆▆▆▆▇▆▆▇▇▅▇▇▅▆▇▄▆▇██
Time into epoch,▁▂▃▄▅▆█▁▂▄▅▆▇▁▂▃▄▆▇█▂▃▄▅▆▇▁▃▄▅▆█▁▂▃▅▆▇█▂


Epoch 100/100: 100%|███████| 30/30 [00:20<00:00,  1.44img/s, Cumulative Dice=[0.9859864711761475], loss (batch)=0.0644]


Angle -10 Test Loss,█▃▂▂▁▂▁▂▂▁▃▂▃▃▃▃▃▃▃▃▃▃▃▄▅▅▅▅▅▄▅▆▅▅▅▆▅▇▇▇
Angle -20 Test Loss,█▄▂▂▁▂▁▂▂▁▃▂▃▃▃▃▃▃▃▃▄▃▃▄▅▅▅▅▅▄▅▆▅▅▅▆▅▇▇▇
Angle -5 Test Loss,█▃▂▁▁▂▁▂▂▁▃▂▃▃▃▃▃▃▃▃▃▃▃▄▅▅▅▅▅▄▅▆▅▅▅▆▅▇▇▇
Angle 0 Test Loss,█▄▂▁▁▁▁▁▂▁▃▂▃▃▃▃▃▃▃▃▃▃▃▄▄▄▅▅▅▄▅▆▅▅▅▆▅▇▇▇
Angle 10 Test Loss,█▃▂▁▁▂▁▂▂▁▃▂▃▃▃▃▃▃▃▃▃▃▃▄▅▄▅▅▅▄▄▆▅▅▅▅▅▇▇▆
Angle 20 Test Loss,█▄▂▂▁▂▁▂▂▁▃▂▃▃▃▃▃▃▃▃▃▃▃▄▅▅▅▅▅▄▅▆▅▅▅▆▅▇▇▇
Angle 5 Test Loss,█▃▂▁▁▁▁▂▂▁▃▂▃▃▃▃▃▃▃▃▃▃▃▄▅▄▅▅▅▄▅▆▅▅▅▆▅▇▇▆
Equivariance Weight,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Random Equivariance Error,▁▁▁▁▁▅▃▂▇▅▅▆▅▄▄▅▃▃▅▆▄▅▅▄▆▅▄▆▆▂▇▆▄█▇▂▄▇▆▄
Test Equivariance Error,▁▁▁▁▂▂▁▂▂▂▂▃▂▆▃▂▅▃▃▂▃▃▃▃▅▄▄▅▅█▆▅█▅▆▇▅▆▅▆
Time into epoch,▁▂▃▅▆▆█▁▃▄▅▆█▁▃▃▅▆▇█▂▃▄▆▇█▁▃▄▅▆█▁▂▃▆▆▇█▂


Epoch 100/100: 100%|███████| 30/30 [00:21<00:00,  1.40img/s, Cumulative Dice=[0.9850479960441589], loss (batch)=0.0612]


Angle -10 Test Loss,█▅▂▁▁▁▁▂▂▂▂▃▂▃▂▃▂▂▃▃▃▂▂▃▃▃▄▄▄▅▄▄▄▄▄▅▅▅▅▅
Angle -20 Test Loss,█▅▂▁▂▁▁▂▁▂▂▃▂▂▂▂▁▂▂▂▃▂▂▃▃▃▄▄▄▄▄▄▄▃▄▅▅▅▅▅
Angle -5 Test Loss,█▄▂▁▁▁▁▂▂▂▂▃▂▃▂▃▂▂▃▃▃▂▂▃▄▄▄▄▄▅▄▄▄▄▄▅▅▅▅▅
Angle 0 Test Loss,█▄▂▁▁▁▁▂▁▂▂▃▂▂▂▃▂▂▃▂▃▂▂▃▃▃▄▄▄▄▄▄▄▄▄▅▄▅▅▅
Angle 10 Test Loss,█▄▂▁▁▁▁▂▂▂▂▃▂▂▂▃▂▂▃▃▃▂▂▃▃▃▄▄▄▄▄▄▃▄▄▅▅▅▅▅
Angle 20 Test Loss,█▅▂▁▂▁▁▂▁▁▂▃▂▂▂▂▁▂▂▃▃▂▂▃▃▃▄▄▄▅▄▄▃▄▄▅▅▅▅▅
Angle 5 Test Loss,█▄▂▁▁▁▁▂▂▂▂▃▂▂▂▃▂▂▃▃▃▂▂▃▃▃▄▄▄▅▄▄▄▄▄▅▅▅▅▅
Equivariance Weight,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Random Equivariance Error,▂▁▁▁▂▂▁▂▅▅▁▆▅▁▂▅▃▅▄▅▇▅▅▃█▅▄▇▅▂▇▆▂▆▅▁▄▅▄▄
Test Equivariance Error,▁▁▂▁▁▂▂▂▂▂▂▂▂▄▃▂▄▂▃▃▃▃▃▃▄▃▄▅▅▆▅▄▇▅▅█▅▆▅▅
Time into epoch,▁▂▃▅▆▆█▁▂▄▅▆█▁▂▃▄▆▇█▂▃▄▆▆█▁▃▄▅▆█▁▂▃▅▅▇█▂


...

Epoch 100/100: 100%|███████| 30/30 [00:20<00:00,  1.49img/s, Cumulative Dice=[0.9852439165115356], loss (batch)=0.0837]


Angle -10 Test Loss,█▄▃▃▂▁▂▃▁▁▁▃▃▃▃▃▄▃▃▃▃▄▄▄▄▅▅▅▆▅▄▄▄▅▅▆▇▆▇▆
Angle -20 Test Loss,█▄▃▃▂▁▂▄▁▁▂▃▃▃▄▄▄▄▃▄▃▄▄▄▄▅▅▅▆▅▅▄▅▅▅▆▇▆▇▆
Angle -5 Test Loss,█▄▃▂▂▁▂▃▁▁▂▃▃▃▃▃▄▃▃▃▃▃▄▄▄▅▅▅▆▅▅▄▄▅▅▆▆▆▆▆
Angle 0 Test Loss,█▄▂▂▁▁▂▂▁▁▂▂▂▃▃▃▃▃▃▃▃▃▄▃▄▄▅▅▆▅▄▄▄▅▅▆▆▅▆▆
Angle 10 Test Loss,█▄▃▃▂▁▂▃▁▁▁▃▃▃▃▃▄▃▂▃▃▃▄▃▄▅▅▅▆▅▄▄▄▅▅▆▆▆▆▆
Angle 20 Test Loss,█▄▃▃▂▁▂▃▁▂▁▃▃▃▄▃▄▄▃▃▃▄▄▄▄▅▅▅▆▅▅▄▄▅▅▆▇▆▇▆
Angle 5 Test Loss,█▄▂▂▂▁▂▃▁▁▁▂▃▃▃▃▄▃▂▃▃▃▄▃▄▅▅▅▆▅▄▄▄▅▅▆▆▆▆▆
Equivariance Weight,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Random Equivariance Error,▂▁▁▂▃▅▄▆▄▂▃▇▆▂▆▆▂▆▆▅▆█▆▅▇▅▅▆▅▂▇▇▁▆▆▂▇▇▇▆
Test Equivariance Error,▁▁▁▁▁▂▂▂▂▂▂▃▂▅▃▃▄▂▃▂▃▃▃▃▅▃▄▆▅█▆▅▇▅▅█▆▆▆▆
Time into epoch,▁▂▃▄▅▆█▁▂▄▅▆▇▁▃▃▄▆▇█▂▃▄▅▆█▁▂▄▅▆▇▁▂▄▄▆██▂


Epoch 100/100: 100%|███████| 30/30 [00:19<00:00,  1.50img/s, Cumulative Dice=[0.9856855869293213], loss (batch)=0.0872]


Angle -10 Test Loss,█▄▂▁▂▁▂▁▁▂▂▂▂▂▃▃▂▂▃▂▃▂▃▄▃▄▅▅▄▅▄▅▅▅▅▅▅▅▆▆
Angle -20 Test Loss,█▅▂▁▂▂▂▂▁▂▃▂▂▃▃▃▂▃▄▂▃▃▃▄▃▅▅▅▄▅▄▅▅▅▆▆▅▅▆▆
Angle -5 Test Loss,█▄▂▁▁▁▂▁▁▂▂▂▂▂▃▃▂▂▃▂▃▂▃▄▃▄▅▅▄▅▄▅▅▅▅▅▅▅▆▆
Angle 0 Test Loss,█▄▂▂▁▁▂▁▁▂▂▂▂▂▂▂▂▂▃▂▂▂▃▃▃▄▄▅▄▅▄▅▄▄▅▅▅▅▅▆
Angle 10 Test Loss,█▄▂▁▂▁▂▁▁▂▂▂▂▂▃▃▂▂▃▂▂▂▃▃▃▄▄▅▄▅▄▅▅▄▅▅▅▅▅▆
Angle 20 Test Loss,█▄▂▁▂▁▂▂▁▂▃▂▂▃▃▃▂▃▃▂▃▂▃▄▃▄▅▅▄▅▄▅▅▅▅▅▅▅▆▆
Angle 5 Test Loss,█▄▂▁▁▁▂▁▁▂▂▂▂▂▃▂▂▂▃▂▂▂▃▄▃▄▄▅▄▅▄▅▅▄▅▅▅▅▆▆
Equivariance Weight,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Random Equivariance Error,▂▁▁▁▁▂▁▁▃▁▂▆▂▁▃▃▁▃▅▅▅▃▄▅█▂▄▇▆▁█▇▂▅▇▂▄▆▅▆
Test Equivariance Error,▁▂▁▁▁▂▁▁▂▂▂▃▂▅▂▂▅▂▂▃▂▃▃▃▄▃▄▄▄▇▅▄▆▄▅█▅▅▄▅
Time into epoch,▁▂▃▄▅▆█▁▃▄▅▆▇▁▃▃▄▆▇█▂▃▄▅▆▇▁▂▄▅▇▇▁▂▄▄▅▇█▂


Testing round, angle = -20:  20%|██████████▍                                         | 6/30 [00:01<00:06,  3.84batch/s]

In [ ]:
for j in range(10):
    args['eqweight'] = 10
    args['augmented'] = 'rangle'
    model = UNet(args['in_channels'], args['classes'], **args).to(device=device)
    model = model.to(memory_format=torch.channels_last)
    train_loader = config_data(HeLa=True, Oxford=False, **args)
    train_model(model, device, train_loader, **args)

wandb: Currently logged in as: jjkjjk23. Use `wandb login --relogin` to force relogin


Epoch 1/50:   0%|                                                                              | 0/30 [00:00<?, ?img/s]C:\Users\jjkjj\anaconda3\envs\equivariant\Lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epoch 50/50: 100%|██████████| 30/30 [00:51<00:00,  1.71s/img, Cumulative Dice=[0.9619008898735046], loss (batch)=0.173]


Angle -10 Test Loss,█▆▄▃▂▁▁▂▂▂▁▁▂▂▁▁▂▂▁▂▁▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂
Angle -20 Test Loss,█▆▅▃▁▁▁▂▂▂▂▁▂▂▁▂▂▂▁▂▁▁▂▂▂▂▂▂▃▂▂▂▃▂▃▂▂▂▂▂
Angle -5 Test Loss,█▆▄▃▂▁▁▂▂▂▂▁▂▂▁▁▂▂▁▂▁▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂
Angle 0 Test Loss,█▆▄▃▂▂▁▂▂▂▁▁▁▂▁▁▂▂▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
Angle 10 Test Loss,█▆▄▃▂▁▁▂▂▂▂▁▂▂▁▁▂▂▁▂▁▂▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂
Angle 20 Test Loss,█▆▅▃▁▁▁▂▂▂▂▁▂▂▁▁▂▂▁▂▁▁▁▂▂▂▂▂▃▂▂▂▂▂▃▂▂▁▂▂
Angle 5 Test Loss,█▆▄▃▂▁▁▂▂▂▂▁▂▂▁▁▂▂▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
Equivariance Weight,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Random Equivariance Error,▇▂▂▁▁▁▁▁▁▁▁▁▁▁▁▃▂▂▂▃▁▁▂▁▃▄▁▄▄▄▁▁▄▄▂█▂▁▃▅
Test Equivariance Error,▅▃▃▃▂▄▄▂▃▄▁▄▄▅▄▄▅▅▄▄▅▆▆▅▅▅▄▆▆█▇▆▇▇▇▇▇▇▆█
Time into epoch,▄▅▆▃▃▃█▁▁▆▆▇▄▄▁▂▃▇██▅▆▆▃▃▄▁▁▆▆▇▄▅▅▂▃▃▇█▅


Epoch 50/50: 100%|██████████| 30/30 [00:53<00:00,  1.80s/img, Cumulative Dice=[0.9630340337753296], loss (batch)=0.176]


Angle -10 Test Loss,█▆▄▃▁▁▂▂▂▁▁▂▂▃▂▂▂▂▁▂▂▂▂▂▂▂▂▃▂▃▂▃▂▂▃▂▂▂▃▂
Angle -20 Test Loss,█▆▄▃▁▁▂▃▃▂▁▂▂▃▂▂▃▂▁▂▂▂▃▃▂▂▃▃▂▄▂▃▂▂▃▂▂▂▄▂
Angle -5 Test Loss,█▅▄▃▁▁▂▂▂▁▁▂▂▂▁▂▂▁▁▂▂▂▂▂▂▂▂▃▂▃▂▃▂▂▃▂▂▂▃▂
Angle 0 Test Loss,█▆▄▃▂▁▂▁▂▁▁▂▁▂▁▁▂▁▁▂▂▂▂▂▂▂▂▃▂▃▂▃▂▂▃▂▂▂▃▂
Angle 10 Test Loss,█▆▄▃▁▁▂▂▂▁▁▂▂▂▂▂▂▁▁▂▂▂▂▂▂▂▂▃▂▃▂▃▂▂▃▂▂▂▃▂
Angle 20 Test Loss,█▆▄▃▁▂▂▃▃▂▁▂▂▃▂▂▃▂▁▂▂▂▃▃▂▂▃▃▂▃▂▃▂▂▃▂▂▃▄▂
Angle 5 Test Loss,█▅▄▃▁▁▁▂▂▁▁▂▂▂▁▁▂▁▁▂▂▂▂▂▂▂▂▃▂▃▂▃▂▂▃▂▂▂▃▂
Equivariance Weight,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Random Equivariance Error,▃▁▁▁▁▁▁▁▁▁▁▁▅▃▆▆▅▅▅▆▆▆▅▁▄▆▂▅▃▇▅▆██▆▇▇▆▅█
Test Equivariance Error,▂▂▁▁▁▂▂▂▂▃▁▃▃▄▄▄▅▄▄▄▆▅▄▄▅▅▄▅▆█▅▆█▆▆▇▇▆▅▆
Time into epoch,▄▅▅▂▃▃█▁▁▆▆▇▄▄▁▂▃▇██▅▆▆▃▃▄▁▁▆▆▇▄▄▅▂▃▃▇█▅


Epoch 50/50: 100%|██████████| 30/30 [00:53<00:00,  1.78s/img, Cumulative Dice=[0.9617347121238708], loss (batch)=0.224]


Angle -10 Test Loss,█▆▅▃▂▂▂▂▂▂▁▁▂▃▂▁▁▂▂▂▁▂▁▂▂▂▂▂▂▃▂▂▂▂▂▂▃▂▃▂
Angle -20 Test Loss,█▆▅▃▃▂▂▂▂▂▁▁▂▃▂▂▁▂▂▂▁▂▂▂▂▃▃▂▂▃▂▂▃▂▂▃▃▂▃▃
Angle -5 Test Loss,█▆▅▃▂▂▂▂▂▂▁▁▂▂▂▁▁▁▂▂▁▂▁▂▂▂▂▂▂▃▂▂▂▂▂▂▃▂▃▂
Angle 0 Test Loss,█▆▅▄▂▂▂▁▂▂▁▁▂▂▂▁▁▁▂▂▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▃▂▂▂
Angle 10 Test Loss,█▆▅▃▂▂▂▂▂▂▁▁▂▃▂▁▁▁▂▂▁▂▁▂▂▂▂▂▂▃▂▂▂▂▂▂▂▂▂▂
Angle 20 Test Loss,█▆▅▃▃▂▂▂▂▂▁▁▂▃▂▂▁▂▂▂▁▂▁▂▂▂▂▂▂▃▂▂▂▂▂▃▂▂▃▂
Angle 5 Test Loss,█▆▅▃▂▂▂▂▂▂▁▁▂▂▂▁▁▁▂▂▁▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
Equivariance Weight,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Random Equivariance Error,▄▂▁▁▁▁▁▁▁▁▁▃▁▂▂▁▂▁▆▂▃▃▁▁▂▃▁▂▃▄▂▂▅▂▂█▅█▅▇
Test Equivariance Error,▃▃▂▂▁▃▃▁▄▃▂▃▄▄▄▄▅▅▅▄▅▄▅▃▆▆▃▆▆█▆▆█▇▇▇▇▇▆█
Time into epoch,▄▅▆▃▃▃█▁▁▆▆▇▄▄▁▂▃▇██▅▆▆▃▃▄▁▁▆▆▇▄▄▅▂▃▃▇█▅


Testing round, angle = -10:  23%|████████████▏                                       | 7/30 [00:02<00:07,  3.01batch/s]

In [10]:
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=torch.device('cpu')
epsilon=0

sizeup = torchvision.transforms.Resize((550,550), interpolation=TF.InterpolationMode.NEAREST_EXACT)
sizedown=torchvision.transforms.Resize((450,450), interpolation=TF.InterpolationMode.NEAREST_EXACT)
resize=torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.NEAREST_EXACT)
deformation = torch.tensor([[0,0,0],[0,.5,0],[0,0,0]]).to(device=device, dtype=torch.float32)
deformation=torch.unsqueeze(deformation, dim=0)
deformation= torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.BICUBIC)(deformation)
ydeformation =torch.full([1,512,512], 0, device=device, dtype=torch.float32)
deformation = torch.stack([deformation, ydeformation], dim=3)
deform = lambda tensor : TF.elastic_transform(tensor, deformation, TF.InterpolationMode.NEAREST, 0.0)

def rotate(angle):
    return lambda inputs : torchvision.transforms.functional.rotate(inputs, angle)


upscale=torchvision.transforms.Compose([sizeup,resize])
etransform=torchvision.transforms.Compose([sizedown,resize])
shrinkcrop=etrainer.compose(etrainer.pad24,sizedown)

def shift(x, shiftnum=1, axis=-1):
    x=torch.transpose(x, axis, -1)
    if shiftnum == 0:
        padded = x
    elif shiftnum > 0:
        #paddings = (0, shift, 0, 0, 0, 0)
        paddings = [0 for j in range(2*len(tuple(x.shape)))]
        paddings[1]=shiftnum
        paddings=tuple(paddings)
        padded = nn.functional.pad(x[..., shiftnum:], paddings)
    elif shiftnum < 0:
        #paddings = (-shift, 0, 0, 0, 0, 0)
        paddings = [0 for j in range(2*len(tuple(x.shape)))]
        paddings[0]=-shiftnum
        paddings=tuple(paddings)
        padded = nn.functional.pad(x[..., :shiftnum], paddings)
    else:
        raise ValueError
    return torch.transpose(padded, axis,-1)
def randshift(x):
    shiftnum = random.randint(-6,6)
    axis = random.randint(-2,-1)
    return shift(x, shiftnum, axis)
#This is for scaling
#efunctions=[[etransform, etransform, epsilon], [upscale,upscale,epsilon]] 
efunctions=[[torchvision.transforms.RandomRotation(10), torchvision.transforms.RandomRotation(10),epsilon]]
#efunctions=efunctions+[[lambda x : shift(x, shiftnum, axis), lambda x : shift(x, shiftnum, axis), epsilon] for shiftnum in range(-1,1,2) for axis in range(-1,1,2)]+[[etransform, etransform, epsilon], [upscale,upscale,epsilon]]
#efunctions += [[randshift, randshift, epsilon]]
#efunctions = efunctions+[[deform,deform,epsilon]]
#efunctions = efunctions + [[rotate(90), rotate(90),0]]
#efunctions = efunctions + [[torchvision.transforms.ElasticTransform(interpolation=TF.InterpolationMode.NEAREST), torchvision.transforms.ElasticTransform(interpolation=TF.InterpolationMode.NEAREST), epsilon]]
"""
for x in range(-1,1):
    for y in range(-1,1):
        for i in range(3):
            for j in range(3):
                for a in range(3):
                    for b in range(3):
                        deformationx = torch.tensor([[x if i==k else 0 for k in range(3)] if j==l else [0,0,0] for l in range(3)]).to(device=device, dtype=torch.float32)
                        deformationy = torch.tensor([[y if a==k else 0 for k in range(3)] if b==l else [0,0,0] for l in range(3)]).to(device=device, dtype=torch.float32)
                        deformationx=torch.unsqueeze(deformationx, dim=0)
                        deformationy=torch.unsqueeze(deformationy, dim=0)
                        deformationx= torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.BICUBIC)(deformationx)
                        deformationy= torchvision.transforms.Resize((512,512), interpolation=TF.InterpolationMode.BICUBIC)(deformationy)
                        deformation = torch.stack([deformationx, deformationy], dim=3)
                        deform = lambda tensor : TF.elastic_transform(tensor, deformation, TF.InterpolationMode.NEAREST, 0.0)
                        efunctions.append([deform,deform,0])
"""
args2 = {'epochs' : 100,
        'batch_size' : 1,
        'amp' : True,
        'bilinear' : False,
        'classes' : 1,
        'learning_rate' : 1e-6,
        'load': False,
        #'load' : "C:\\Users\\jjkjj\\Equivariant\\EquivariantUNet\\bumbling-sponge-27_checkpoints\\checkpoint_epoch121.pth",
        'class weights' : [1,1,3],
        'epochbreaks' : False,
        'break_length' : 5,
        'etransforms' : efunctions,
        'equivariance_measure' : 'l1',
        'equivariant' : False,
        'eqerror' : False,
        'augmented' : 'rangle',
        'Linf' : False,
        'eqweight' : 100,
        'n' : 1,
        'debugging' : False,
        'in_channels' : 1,
        'wandb_project' : 'HeLa EUNet',
        'test_on_epoch_end' : True,
        'test augmented' : 'True no identity',
        'test augment' : 'fixed rotations',
        'save_checkpoint' : True,
        'eqweight_scheduler' : False,
        'eqweight_decay' : 1.1,
        'lr_scheduler' : 'cyclic',
        'min_lr' : 1e-9,
        'max_lr' : 1.5e-4,
        'product_loss' : False
       }
if args['equivariant'] != args['eqerror']:
    print('Equivariant and eqerror are different are you sure?')

model = UNet(args['in_channels'], args['classes'], **args).to(device=device)
model = model.to(memory_format=torch.channels_last)

#model = UNetLightning(args['in_channels'], args['classes'], **args).to(device=device)


if args['load']:
    state_dict = torch.load(args['load'], map_location=device)
    #del state_dict['mask_values']
    model.load_state_dict(state_dict)
    logging.info(f'Model loaded from {args["load"]}')
print(model.n)
print(model.Linf)
#print(model.state_dict())


C:\Users\jjkjj\anaconda3\envs\equivariant\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


1
False
